<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/algebra_y_trigonometria/blob/main/classes/0_formatos_clase/algebra_and_trigonometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marco-canas/algebra_y_trigonometria/blob/main/classes/0_formatos_clase/algebra_and_trigonometry.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# El parámetro `d` en un modelo ARIMA   


El parámetro `d` en un modelo ARIMA representa el número de diferencias no estacionales requeridas para hacer que la serie temporal sea estacionaria. En otras palabras, `d` indica cuántas veces se debe restar el valor anterior de la serie para eliminar tendencias o patrones a largo plazo y estabilizar la media de la serie.  

# Práctica inicial para aprender el concepto de número de diferencias d para estabilizar una serie polinomial  

¡Listo! Te dejé una práctica completa y reproducible para aprender a determinar el **orden de diferenciación $d$** que estabiliza una serie polinomial $f(t)=t^n$ con $n=1,2,3,4,5$.



# Qué incluye la práctica

1. **Tabla de diferencias finitas** para $f(t)=t^3$ (útil para trabajo en clase).
2. **Resumen automático** que compara el $d$ teórico (que es $n$) con el $d$ estimado por el algoritmo (detecta cuando $\Delta^k f$ se vuelve constante).
3. **Gráficos** (uno por figura) de:

   * Serie original $f(t)=t^n$.
   * Serie diferenciada de orden $d=n$ mostrando que es constante.
4. **Archivos descargables** para tus estudiantes:

   * [Descargar tabla de diferencias $t^3$](sandbox:/mnt/data/tabla_diferencias_t3.csv)
   * [Descargar resumen $d$ para $n=1..5$](sandbox:/mnt/data/resumen_d_f_t_n.csv)



# Idea clave (para la guía)

* Para una serie **polinomial** $f(t)=t^n$ con paso 1,

  $$
  \Delta^n f(t) = n! \quad (\text{constante}), \qquad \Delta^{n+1} f(t)=0.
  $$
* Por tanto, el **$d$ mínimo** que elimina la tendencia es **$d=n$**.

# Sugerencia de actividades

* Pide a los estudiantes **llenar a mano** (o con Python) la tabla de $f(t)=t^3$ y sus diferencias hasta $\Delta^4$.
* Que **predigan** el $d$ para $n=4,5$ y lo **verifiquen** con los gráficos y la columna “$d\_estimado\_por\_var≈0$”.
* Extiende con ruido ligero $f(t)=t^n+\varepsilon_t$ y discute por qué a veces conviene **no excederse** en $d$ (sobre-diferenciación).

Si quieres, te genero una versión con **ruido** y chequeos de **autocorrelación** para practicar criterios prácticos en datos reales.


In [2]:
# Práctica: Determinación del parámetro d (orden de diferenciación) para series polinomiales f(t)=t^n
#
# Este cuaderno genera datos artificiales f(t)=t^n, aplica diferencias de orden k,
# muestra tablas de diferencias finitas y resume el d mínimo que estabiliza (vuelve constante) la serie.
#
# Reglas de gráficos exigidas por el sistema: usar matplotlib, una figura por gráfico, sin estilos ni colores específicos.

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt


def serie_polinomial(n:int, T:int=30, start:int=0):
    t = np.arange(start, start+T+1)  # T+1 puntos para que queden T puntos luego de 1 diferencia, etc.
    y = t**n
    return t, y


In [3]:
serie_polinomial(3, T=10, start=1)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([   1,    8,   27,   64,  125,  216,  343,  512,  729, 1000, 1331]))

In [4]:

def diferencia_k(y: np.ndarray, k:int):
    z = y.copy()
    for _ in range(k):
        z = np.diff(z, n=1)
    return z


In [7]:
diferencia_k(np.array([1, 8, 27, 64, 125]), 3)  # diferencias de f(t)=t^3, t=1..5

array([6, 6])

In [ ]:

def d_minimo_para_constante(y: np.ndarray, max_k:int=10, tol:float=1e-12):
    # Encuentra el menor k tal que la serie diferenciada es (numéricamente) constante
    for k in range(0, max_k+1):
        z = diferencia_k(y, k)
        if len(z) <= 1:
            # si queda 0 o 1 punto, la consideramos constante por construcción
            return k
        if np.var(z) < tol:
            return k
    return None

# 1) Tabla de diferencias para n=3 (ejemplo docente)
t, y = serie_polinomial(n=3, T=10, start=0)
tabla = pd.DataFrame({"t": t, "f(t)=t^3": y})
for k in range(1, 6):  # hasta 5 diferencias
    col_prev = y if k==1 else tabla[f"Δ^{k-1} f"]
    tabla[f"Δ^{k} f"] = np.append(np.diff(col_prev), [np.nan])

display_dataframe_to_user("Tabla de diferencias finitas para f(t)=t^3", tabla)

# 2) Resumen para n=1..5: d mínimo teórico vs detectado
resumen_rows = []
for n in range(1, 6):
    t, y = serie_polinomial(n=n, T=30)
    d_estimado = d_minimo_para_constante(y, max_k=10)
    resumen_rows.append({
        "n": n,
        "d_mínimo_teórico (n)": n,
        "d_estimado_por_var≈0": d_estimado,
        "Δ^n f(t) esperado": f"{math.factorial(n)} (constante)",
        "Δ^{n+1} f(t) esperado": "0 (todos ceros)"
    })
resumen = pd.DataFrame(resumen_rows)
display_dataframe_to_user("Resumen d para f(t)=t^n, n=1..5", resumen)

# 3) Gráficos ilustrativos: para cada n, graficar la serie original y la serie tras d=n diferencias
for n in range(1, 6):
    t, y = serie_polinomial(n=n, T=30)
    plt.figure(figsize=(8,4))
    plt.plot(t, y, linewidth=2)
    plt.title(f"Serie original f(t)=t^{n}")
    plt.xlabel("t")
    plt.ylabel("f(t)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    z = diferencia_k(y, n)          # d = n
    t_d = np.arange(t[0]+n, t[-1]+1)  # eje temporal tras n diferencias
    plt.figure(figsize=(8,4))
    plt.plot(t_d, z, linewidth=2)
    plt.title(f"Diferenciada Δ^{n} f(t)=t^{n}  →  constante ≈ {z[0] if len(z)>0 else 'NA'}")
    plt.xlabel("t")
    plt.ylabel(f"Δ^{n} f(t)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# 4) Guardar los resultados clave como CSV para práctica
tabla.to_csv("/mnt/data/tabla_diferencias_t3.csv", index=False)
resumen.to_csv("/mnt/data/resumen_d_f_t_n.csv", index=False)

print("Archivos creados:")
print("- /mnt/data/tabla_diferencias_t3.csv")
print("- /mnt/data/resumen_d_f_t_n.csv")


In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from caas_jupyter_tools import display_dataframe_to_user

def serie_polinomial(n:int, T:int=30, start:int=0):
    t = np.arange(start, start+T+1)
    y = t**n
    return t, y

def diferencia_k(y: np.ndarray, k:int):
    z = y.copy()
    for _ in range(k):
        z = np.diff(z, n=1)
    return z

def d_minimo_para_constante(y: np.ndarray, max_k:int=10, tol:float=1e-12):
    for k in range(0, max_k+1):
        z = diferencia_k(y, k)
        if len(z) <= 1:
            return k
        if np.var(z) < tol:
            return k
    return None

# 1) Tabla de diferencias para n=3 (ejemplo)
t, y = serie_polinomial(n=3, T=10, start=0)
tabla = pd.DataFrame({"t": t, "f(t)=t^3": y})
for k in range(1, 6):
    col_prev = y if k==1 else tabla[f"Δ^{k-1} f"]
    tabla[f"Δ^{k} f"] = np.append(np.diff(col_prev), [np.nan])

display_dataframe_to_user("Tabla de diferencias finitas para f(t)=t^3", tabla)

# 2) Resumen para n=1..5
resumen_rows = []
for n in range(1, 6):
    t, y = serie_polinomial(n=n, T=30)
    d_estimado = d_minimo_para_constante(y, max_k=10)
    resumen_rows.append({
        "n": n,
        "d_mínimo_teórico (n)": n,
        "d_estimado_por_var≈0": d_estimado,
        "Δ^n f(t) esperado": f"{math.factorial(n)} (constante)",
        "Δ^{n+1} f(t) esperado": "0 (todos ceros)"
    })
resumen = pd.DataFrame(resumen_rows)
display_dataframe_to_user("Resumen d para f(t)=t^n, n=1..5", resumen)

# 3) Gráficos: serie original y diferenciada de orden n para n=1..5
for n in range(1, 6):
    t, y = serie_polinomial(n=n, T=30)
    plt.figure(figsize=(8,4))
    plt.plot(t, y, linewidth=2)
    plt.title(f"Serie original f(t)=t^{n}")
    plt.xlabel("t")
    plt.ylabel("f(t)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    z = diferencia_k(y, n)
    t_d = np.arange(t[0]+n, t[-1]+1)
    plt.figure(figsize=(8,4))
    plt.plot(t_d, z, linewidth=2)
    plt.title(f"Diferenciada Δ^{n} f(t)=t^{n}  →  constante ≈ {z[0] if len(z)>0 else 'NA'}")
    plt.xlabel("t")
    plt.ylabel(f"Δ^{n} f(t)")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# 4) Guardar CSV
tabla.to_csv("/mnt/data/tabla_diferencias_t3.csv", index=False)
resumen.to_csv("/mnt/data/resumen_d_f_t_n.csv", index=False)

print("Archivos creados:")
print("- /mnt/data/tabla_diferencias_t3.csv")
print("- /mnt/data/resumen_d_f_t_n.csv")
